In [10]:
import cv2
import pygame
import numpy as np
import pyrealsense2 as rs
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *
from PIL import Image
from matplotlib import pyplot as plt

In [11]:
verts = (
  (1, -1, -1),
  (1, 1, -1),
  (-1, 1, -1),
  (-1, -1, -1),
  (1, -1, 1),
  (1, 1, 1),
  (-1, -1, 1),
  (-1, 1, 1))

edges = (
  (0,1),(0,3),(0,4),
  (2,1),(2,3),(2,7),
  (6,3),(6,4),(6,7),
  (5,1),(5,4),(5,7))

surfaces = (
  (0,1,2,3),
  (3,2,7,6),
  (6,7,5,4),
  (4,5,1,0),
  (1,5,7,2),
  (4,0,3,6))

colors = (
  (1,0,0),
  (0,1,0),
  (0,0,1),
  (1,1,0),
  (0,1,1),
  (1,0,1))    

In [12]:
frame, gray = 0, 0
capture = 0

displayCamera = True
displayInformation = True

projection = True
button = False
loop = True
clock, frameCount = 0, 0

faceType = False
ym = 5.4                                    # display image y margin
rotX, rotY = 0, 0
scale, zScale = 2., 1.                     # opengl display scale, z axis scale

objSx, objSy = 0, 0                         # face object start x, y
objWidth, objHeight = 10, 10                # face object width, height
minDepth, maxDepth, meanDepth = 0, 0, 0     # face object min, max, mean depth 

msg = "..."
flirMsg = "..."
font, fontSize = 0, 18
fRate, size = 0.1, 5
mouseX, mouseY, pmouseX, pmouseY = 0, 0, 0, 0

eye = (0., 0., 10.)
center = (0., 0., 0.)
fps, delay = 30, 1 # 33


In [13]:

def myInit():
    global winWidth, winHeight, objWidth, objHeight
    global font, fontSize, loop, frameCount, clock, winWidth, winHeight
    loop = True
    frameCount = 0

    pygame.init()
    clock = pygame.time.Clock()

    pygame.display.set_caption('jluFace3D Analysis with realSensor')
    screen = pygame.display.set_mode((winWidth, winHeight), DOUBLEBUF|OPENGL) # FULLSCREEN
    font = pygame.font.SysFont("courier new", fontSize, bold=True) #'arial'
    myTexture()


def myTexture():
    image = Image.open("simage1.jpg") #realsense-logo.png
    img_data = np.array(list(image.getdata()), np.uint8)
    updateImage(img_data, image.width, image.height, 1) #dwidth, dheight
    
    glBindTexture(GL_TEXTURE_2D, 1)
    glTexParameteri(GL_TEXTURE_2D,GL_TEXTURE_MAG_FILTER, GL_NEAREST)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_NEAREST)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, 
        image.width, image.height, 0, GL_RGB, GL_UNSIGNED_BYTE, img_data)
    glTexEnvf(GL_TEXTURE_ENV, GL_TEXTURE_ENV_MODE, GL_REPLACE)


def myReshape():
    global projection, size, winWidth, winHeight
    glClearColor (.75, .75, .75, 0.0)
    glViewport(0, 0, winWidth, winHeight)
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluPerspective(60., winWidth/winHeight, .1, 10000.)
    glMatrixMode(GL_MODELVIEW)
    glEnable(GL_DEPTH_TEST)


def updateImage(image, width, height, index):
    glBindTexture(GL_TEXTURE_2D, index)
    # img_data = np.array(list(image)) #, np.uint8)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, width, height, 0, GL_BGR, GL_UNSIGNED_BYTE, image) #img_data
    

def drawText(x, y, text, gray): 
    textSurface = font.render(text, True, (255, 255, 255, 255), (gray, gray, gray, 255))
    textData = pygame.image.tostring(textSurface, "RGBA", True)
    glWindowPos2d(x, y)
    glDrawPixels(textSurface.get_width(), textSurface.get_height(), GL_RGBA, GL_UNSIGNED_BYTE, textData)

    
def myFace():
    if faceType:
        glEnable(GL_TEXTURE_2D)
        glBindTexture(GL_TEXTURE_2D, 1)
        glBegin(GL_QUADS)
        glTexCoord2f(0.0, 1.0) 
        glVertex3f(-1, -1, 1)
        glTexCoord2f(1.0, 1.0) 
        glVertex3f(1, -1, 1)
        glTexCoord2f(1.0, 0.0) 
        glVertex3f(1, 1, 1)
        glTexCoord2f(0.0, 0.0) 
        glVertex3f(-1, 1, 1)
        glEnd()
        glDisable(GL_TEXTURE_2D)

    glBegin(GL_LINES)
    i = 0
    for edge in edges:
        glColor3fv(colors[i%len(colors)])
        for vertex in edge:
            glVertex3fv(verts[vertex])
        i += 1    
    glEnd()


def myDisplay():    
    global eye, center, delay, rotX, rotY, scale
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT) 
    glLoadIdentity()   
    gluLookAt(eye[0], eye[1], eye[2], center[0], center[1], center[2], 0, 1, 0)

    if displayInformation:
        myMessage()
    if displayCamera:
        myCameraTexture()
    
    glRotatef(rotX, 1, 0, 0)
    glRotatef(rotY, 0, 1, 0)
    glScalef(scale, scale, scale)
    
    myFace()
    
    pygame.display.flip()
    pygame.time.wait(delay)


def myQuit():
    pygame.quit()
    cv2.destroyAllWindows()
    

In [14]:
def myMessage():
    global winWidth, winHeight, fontSize, clock   
    global displayCamera, displayInformation, faceType, frameCount

    w, h = 7, 1.4
    glPushMatrix()
    glTranslatef(-8.2, ym-h, 0)
    glColor3f(.3, .3, .3)
    glBegin(GL_QUADS)
    glVertex3f(0, 0, 0)
    glVertex3f(w, 0, 0)
    glVertex3f(w, h, 0)
    glVertex3f(0, h, 0)
    glEnd()
    glPopMatrix()
    xmargin = 40
    ymargin = 40
    gray = 76
    drawText(xmargin, winHeight-ymargin-fontSize, "inform(i)"+str(displayInformation)+" scale(z/x)"+'{:.2f}'.format(scale), gray)
    drawText(xmargin, winHeight-ymargin-2*fontSize, "faceType(f)"+str(faceType)+" camera(c)"+str(displayCamera), gray)
    drawText(xmargin, winHeight-ymargin-3*fontSize, "fps:" + str(int(clock.get_fps()))+"-"+str(frameCount), gray)
    drawText(xmargin, winHeight-ymargin-4*fontSize, msg, gray)


def myCameraTexture():
    global ym, frame, camWidth, camHeight

    w, h = 6.4, 4.8 
    updateImage(frame, camWidth, camHeight, 1)
    glPushMatrix()
    glTranslatef(-8.2, -ym, 0)
    # glTranslatef(8.2-w, -ym, 0)
    # glTranslatef(8.2-w, ym-h, 0)
    glColor3f(.3, .3, .3)
    glEnable(GL_TEXTURE_2D)
    glBindTexture(GL_TEXTURE_2D, 1)
    glBegin(GL_QUADS)
    glTexCoord2f(0.0, 1.0) 
    glVertex3f(0, 0, 0)
    glTexCoord2f(1.0, 1.0) 
    glVertex3f(w, 0, 0)
    glTexCoord2f(1.0, 0.0) 
    glVertex3f(w, h, 0)
    glTexCoord2f(0.0, 0.0) 
    glVertex3f(0, h, 0)
    glEnd()
    glPopMatrix()
    glDisable(GL_TEXTURE_2D)


In [15]:
def myCamera():
    global displayCamera, capture, frame 
    if displayCamera == False :
        return
    if capture : 
        ret, frame = capture.read()
        if ret:
            frame = cv2.flip(frame, 1)

            
def setCamera():
    global capture, fps, msg, camWidth, camHeight, camId
    if camId < 0:
        return    
    capture = cv2.VideoCapture(camId) # camera id    
    if capture:
        capture.set(cv2.CAP_PROP_FPS, fps)
        capture.set(cv2.CAP_PROP_FRAME_WIDTH, camWidth) #1920x1080 1280x720 960x540 640x360
        capture.set(cv2.CAP_PROP_FRAME_HEIGHT, camHeight) #640x480
        msg = "Connect Camera " + str(camId) + " ... "
        print('connect camera ' + str(camId) + ' width: ' + str(camWidth) + ', height: ' + str(camHeight) + ', fps: ' + str(fps))
    else:    
        msg = "No Camera ... "
        print('No Camera ... ')

     
def stopCamera():
    global capture
    if capture:    
        capture.release()
        capture = 0



In [16]:
def myEvent():
    global loop, faceType, rotX, rotY, scale, msg
    global mouseX, mouseY, pmouseX, pmouseY, button
    global displayInformation, displayCamera, frameCount
    frameCount+=1

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            loop = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                loop = False
                msg = "ESC stop..."
            elif event.key == pygame.K_i:
                displayInformation = not displayInformation
                msg = "i key information : " + str(displayInformation)
            elif event.key == pygame.K_c:
                displayCamera = not displayCamera
                msg = "c key displayCamera : " + str(displayCamera)
            elif event.key == pygame.K_f:
                faceType = not faceType
                msg = "f key faceType : " + str(faceType)
            elif event.key == pygame.K_u:
                rotX = rotY = 0
                msg = "u key reset rot"
            elif event.key == pygame.K_z:
                scale *= 1.075
                msg = "z key scale : " + '{:.2f}'.format(scale)
            elif event.key == pygame.K_x:
                scale /= 1.075
                msg = "x key scale : " + '{:.2f}'.format(scale)
        elif event.type == pygame.MOUSEBUTTONDOWN:
            mouseX, mouseY = pygame.mouse.get_pos()
            pmouseX, pmouseY, button = mouseX, mouseY, True
            msg = "mouse down (" + str(mouseX) + "," + str(mouseY)+ ") " + str(button)
        elif event.type == pygame.MOUSEMOTION:
            mouseX, mouseY = pygame.mouse.get_pos()
            if button == True:
                rotX -= (pmouseY - mouseY)*fRate
                rotY -= (pmouseX - mouseX)*fRate
                pmouseX, pmouseY = mouseX, mouseY
        elif event.type == pygame.MOUSEBUTTONUP:
            mouseX, mouseY = pygame.mouse.get_pos()
            button = False
            msg = "mouse up (" + str(mouseX) + "," + str(mouseY)+ ") " + str(button)   
       

In [17]:
def main():
    global loop, fps
    myInit()
    myReshape()
    setCamera()
    while(loop):
        myCamera()
        myDisplay()
        myEvent()    
        clock.tick(fps)
  
    stopCamera()
    myQuit()

In [18]:
camId = 0                     # 1:display log 2:cv2.imshow 3:cv2.imwrite 
fps = 30                     # Flir Image Green frame threshold    
winWidth, winHeight = 1200, 800
camWidth, camHeight = 640, 480 
main()


connect camera 0 width: 640, height: 480, fps: 30
